In [2]:
import tensorflow as tf
import numpy as np

`tf.nn.embedding_lookup`

In [2]:
input_ids = tf.placeholder(tf.int32, shape=[None])
embedding = tf.Variable(np.identity(5, dtype=np.float32))
input_embedding = tf.nn.embedding_lookup(embedding, input_ids)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [3]:
embedding.eval()

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]], dtype=float32)

In [4]:
sess.run(input_embedding, feed_dict={input_ids: [1, 2, 3, 0, 3, 2, 1, 0]})

array([[ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.]], dtype=float32)

In [5]:
input_ids = tf.placeholder(tf.int32)
input_embedding = tf.nn.embedding_lookup(embedding, input_ids)
sess.run(input_embedding, feed_dict={input_ids:[[1, 2], [2, 1], [3, 3], [1, 1]]})

array([[[ 0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.]],

       [[ 0.,  0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.,  0.]],

       [[ 0.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.]]], dtype=float32)

In [6]:
tf.placeholder?

In [1]:
tf.nn.embedding_lookup?

Object `tf.nn.embedding_lookup` not found.


## Graphs & Sessions

- TensorFlow uses a **dataflow graph** to represent your computation in terms of the dependencies between individual operations. 
- This leads to a low-level programming model in which you first define the dataflow graph, then create a TensorFlow **session** to run parts of the graph across a set of local and remote devices.
- In a dataflow graph, the nodes represent units of computation, and the edges represent the data consumed or produced by a computation.
- Dataflow advantages:
  - Parallelism
  - Distributed execution
  - Compilation
  - Portability
  

In [1]:
import tensorflow as tf
tf.Graph?

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


- A `Graph` contains a set of @{tf.Operation} objects, which represent units of computation; and
@{tf.Tensor} objects, which represent the units of data that flow between operations.

- A `Graph` instance supports an arbitrary number of "collections" that are identified by name. For convenience when building a large
graph, collections can store groups of related objects: for
example, the `tf.Variable` uses a collection (named
@{tf.GraphKeys.GLOBAL_VARIABLES}) for
all variables that are created during the construction of a graph. The caller
may define additional collections by specifying a new name.

A tf.Graph contains two relevant kinds of information:
- **Graph structure**. The nodes and edges of the graph, indicating how individual operations are composed together, but not prescribing how they should be used. The graph structure is like assembly code: inspecting it can convey some useful information, but it does not contain all of the useful context that source code conveys.
- **Graph collections**. TensorFlow provides a general mechanism for storing collections of metadata in a tf.Graph. The tf.add_to_collection function enables you to associate a list of objects with a key (where tf.GraphKeys defines some of the standard keys), and tf.get_collection enables you to look up all objects associated with a key. Many parts of the TensorFlow library use this facility: for example, when you create a tf.Variable, it is added by default to collections representing "global variables" and "trainable variables". When you later come to create a tf.train.Saver or tf.train.Optimizer, the variables in these collections are used as the default arguments.

- TensorFlow provides a default graph that is an implicit argument to all API functions in the same context.
- Calling tf.constant(42.0) creates a single tf.Operation that produces the value 42.0, adds it to the default graph, and returns a tf.Tensor that represents the value of the constant.
- Calling tf.matmul(x, y) creates a single tf.Operation that multiplies the values of tf.Tensor objects x and y, adds it to the default graph, and returns a tf.Tensor that represents the result of the multiplication.
- Executing v = tf.Variable(0) adds to the graph a tf.Operation that will store a writeable tensor value that persists between tf.Session.run calls. The tf.Variable object wraps this operation, and can be used like a tensor, which will read the current value of the stored value. The tf.Variable object also has methods such as assign and assign_add that create tf.Operation objects that, when executed, update the stored value. (See Variables for more information about variables.)
- Calling tf.train.Optimizer.minimize will add operations and tensors to the default graph that calculate gradients, and return a tf.Operation that, when run, will apply those gradients to a set of variables.
- Calling most functions in the TensorFlow API merely adds operations and tensors to the default graph, but does not perform the actual computation. nstead, you compose these functions until you have a tf.Tensor or tf.Operation that represents the overall computation--such as performing one step of gradient descent--and then pass that object to a tf.Session to perform the computation.
- A tf.Graph object defines a namespace for the tf.Operation objects it contains. 
  - Each API function that creates a new tf.Operation or returns a new tf.Tensor accepts an optional name argument.
  - tf.constant(42.0, name="answer") creates a new tf.Operation named "answer" and returns a tf.Tensor named "answer:0". If the default graph already contained an operation named "answer", the TensorFlow would append "_1", "_2", and so on to the name, in order to make it unique.
  - The tf.name_scope function makes it possible to add a name scope prefix to all operations created in a particular context. The current name scope prefix is a "/"-delimited list of the names of all active tf.name_scope context managers. If a name scope has already been used in the current context, TensorFlow appens "_1", "_2", and so on.

In [2]:
c_0 = tf.constant(0, name="c")  # => operation named "c"
c_0

<tf.Tensor 'c:0' shape=() dtype=int32>

In [3]:
c_1 = tf.constant(2, name="c")
c_1

<tf.Tensor 'c_1:0' shape=() dtype=int32>

In [4]:
# Name scopes add a prefix to all operations created in the same context.
with tf.name_scope("outer"):
    c_2 = tf.constant(2, name="c")
    # Name scopes nest like paths in a hierarchical file system.
    with tf.name_scope("inner"):
        c_3 = tf.constant(3, name="c")
    # Exiting a name scope context will return to the previous prefix.
    c_4 = tf.constant(4, name="c")  # => operation named "outer/c_1"
    # Already-used name scopes will be "uniquified".
    with tf.name_scope("inner"):
        c_5 = tf.constant(5, name="c") 

In [6]:
c_2, c_3, c_4, c_5

(<tf.Tensor 'outer/c:0' shape=() dtype=int32>,
 <tf.Tensor 'outer/inner/c:0' shape=() dtype=int32>,
 <tf.Tensor 'outer/c_1:0' shape=() dtype=int32>,
 <tf.Tensor 'outer/inner_1/c:0' shape=() dtype=int32>)

Note that tf.Tensor objects are implicitly named after the `tf.Operation` that produces the tensor as output. 
A tensor name has the form `"<OP_NAME>:<i>"` where:

- `"<OP_NAME>"` is the name of the operation that produces it.
- `"<i>"` is an integer representing the index of that tensor among the operation's outputs.

### Placing operations on different devices

- A device specification has the following form:
`/job:<JOB_NAME>/task:<TASK_INDEX>/device:<DEVICE_TYPE>:<DEVICE_INDEX>`

> https://www.tensorflow.org/programmers_guide/graphs

In [ ]:
# if you are running in a single-machine configuration with a single GPU, you might use tf.device to pin some operations to the CPU and GPU:

# Operations created outside either context will run on the "best possible"
# device. For example, if you have a GPU and a CPU available, and the operation
# has a GPU implementation, TensorFlow will choose the GPU.
weights = tf.random_normal(...)

with tf.device("/device:CPU:0"):
  # Operations created in this context will be pinned to the CPU.
  img = tf.decode_jpeg(tf.read_file("img.jpg"))

with tf.device("/device:GPU:0"):
  # Operations created in this context will be pinned to the GPU.
  result = tf.matmul(weights, img)

### Tensor-like objects
For convenience, these functions will accept a tensor-like object in place of a tf.Tensor, and implicitly convert it to a `tf.Tensor` using the `tf.convert_to_tensor` method. Tensor-like objects include elements of the following types:
- tf.Tensor
- tf.Variable
- numpy.ndarray
- list (and lists of tensor-like objects)
- Scalar Python types: bool, float, int, str

> **By default, TensorFlow will create a new tf.Tensor each time you use the same tensor-like object**. If the tensor-like object is large (e.g. a numpy.ndarray containing a set of training examples) and you use it multiple times, you may run out of memory. To avoid this, manually call tf.convert_to_tensor on the tensor-like object once and use the returned tf.Tensor instead.

In [9]:
import numpy as np

def my_func(arg):
  arg = tf.convert_to_tensor(arg, dtype=tf.float32)
  return tf.matmul(arg, arg) + arg

# The following calls are equivalent.
value_1 = my_func(tf.constant([[1.0, 2.0], [3.0, 4.0]]))
value_2 = my_func([[1.0, 2.0], [3.0, 4.0]])
value_3 = my_func(np.array([[1.0, 2.0], [3.0, 4.0]], dtype=np.float32))

value_1, value_2, value_3

(<tf.Tensor 'add_3:0' shape=(2, 2) dtype=float32>,
 <tf.Tensor 'add_4:0' shape=(2, 2) dtype=float32>,
 <tf.Tensor 'add_5:0' shape=(2, 2) dtype=float32>)

### tf.Session 
- TensorFlow uses the tf.Session class to represent a connection between the client program---typically a Python program
- A tf.Session object provides access to devices in the local machine, and remote devices using the distributed TensorFlow runtime. It also **caches** information about your tf.Graph so that you can efficiently run the same computation multiple times.
- Since a tf.Session owns physical resources (such as GPUs and network connections), it is typically used as a context manager (in a with block) that automatically closes the session when you exit the block. It is also possible to create a session without using a with block, but you should explicitly call tf.Session.close when you are finished with it to free the resources.
-  Higher-level APIs such as `tf.train.MonitoredTrainingSession` or `tf.estimator.Estimator` will create and manage a tf.Session for you. 
- By default, a new tf.Session will be bound to---and only able to run operations in---the current default graph.
- [tf.ConfigProto](https://github.com/tensorflow/tensorflow/blob/r1.4/tensorflow/core/protobuf/config.proto)
- The `tf.Session.run` method is the main mechanism for running a tf.Operation or evaluating a tf.Tensor. You can pass one or more tf.Operation or tf.Tensor objects to tf.Session.run, and TensorFlow will execute the operations that are needed to compute the result.
- `tf.Session.run` requires you to specify a list of fetches, which determine the return values, and may be a tf.Operation, a tf.Tensor, or a tensor-like type such as tf.Variable. These fetches determine what subgraph of the overall tf.Graph must be executed to produce the result.
- tf.Session.run also optionally takes a dictionary of feeds, which is a mapping from tf.Tensor objects (typically tf.placeholder tensors) to values (typically Python scalars, lists, or NumPy arrays) that will be substituted for those tensors in the execution.

In [12]:
# Create a default in-process session.
with tf.Session() as sess:
    print(sess)

In [14]:
# Create a remote session.
with tf.Session("grpc://example.org:2222"):
    # ...
    pass

In [16]:
x = tf.constant([[37.0, -23.0], [1.0, 4.0]])
w = tf.Variable(tf.random_uniform([2, 2]))
y = tf.matmul(x, w)
output = tf.nn.softmax(y)
init_op = w.initializer

with tf.Session() as sess:
  # Run the initializer on `w`.
  sess.run(init_op)

  # Evaluate `output`. `sess.run(output)` will return a NumPy array containing
  # the result of the computation.
  print(sess.run(output))

  # Evaluate `y` and `output`. Note that `y` will only be computed once, and its
  # result used both to return `y_val` and as an input to the `tf.nn.softmax()`
  # op. Both `y_val` and `output_val` will be NumPy arrays.
  y_val, output_val = sess.run([y, output])

[[0.00366294 0.99633706]
 [0.4479277  0.5520723 ]]


In [26]:
# Define a placeholder that expects a vector of three floating-point values,
# and a computation that depends on it.
x = tf.placeholder(tf.float32, shape=[3])
y = tf.square(x)

with tf.Session() as sess:
    # Feeding a value changes the result that is returned when you evaluate `y`.
    print(sess.run(y, {x: [1.0, 2.0, 3.0]}))  # => "[1.0, 4.0, 9.0]"
    print(sess.run(y, {x: [0.0, 0.0, 5.0]}))  # => "[0.0, 0.0, 25.0]"

  # Raises `tf.errors.InvalidArgumentError`, because you must feed a value for
  # a `tf.placeholder()` when evaluating a tensor that depends on it.
    try:
        sess.run(y)
    except Exception as e:
        print(type(e))

  # Raises `ValueError`, because the shape of `37.0` does not match the shape
  # of placeholder `x`.
    try: 
        sess.run(y, {x: 37.0})
    except ValueError as e:
        print(type(e))
    print('DONE')

[1. 4. 9.]
[ 0.  0. 25.]
<class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>
<class 'ValueError'>
DONE


In [31]:
import pprint
y = tf.matmul([[37.0, -23.0], [1.0, 4.0]], tf.random_uniform([2, 2]))

with tf.Session() as sess:
  # Define options for the `sess.run()` call.
  options = tf.RunOptions()
  options.output_partition_graphs = True
  options.trace_level = tf.RunOptions.FULL_TRACE

  # Define a container for the returned metadata.
  metadata = tf.RunMetadata()

  sess.run(y, options=options, run_metadata=metadata)

  # Print the subgraphs that executed on each device.
  pprint.pprint(metadata.partition_graphs)

[node {
  name: "MatMul_10/a"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
          dim {
            size: 2
          }
        }
        tensor_content: "\000\000\024B\000\000\270\301\000\000\200?\000\000\200@"
      }
    }
  }
}
node {
  name: "random_uniform_4/shape"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 2
          }
        }
        tensor_content: "\002\000\000\000\002\000\000\000"
      }
    }
  }
}
node {
  name: "random_uniform_4/RandomUniform"
  op: "RandomUniform"
  input: "random_uniform_4/shape"
 

In [32]:
  # Print the timings of each operation that executed.
  pprint.pprint(metadata.step_stats)

dev_stats {
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  node_stats {
    node_name: "_SOURCE"
    all_start_micros: 1516257747486266
    op_start_rel_micros: 2
    op_end_rel_micros: 2
    all_end_rel_micros: 6
    memory {
      allocator_name: "cpu"
    }
    timeline_label: "_SOURCE = NoOp()"
    scheduled_micros: 1516257747486227
    memory_stats {
    }
  }
  node_stats {
    node_name: "MatMul_10/a"
    all_start_micros: 1516257747486277
    all_end_rel_micros: 4
    memory {
      allocator_name: "cpu"
    }
    output {
      tensor_description {
        dtype: DT_FLOAT
        shape {
          dim {
            size: 2
          }
          dim {
            size: 2
          }
        }
        allocation_description {
          requested_bytes: 16
          allocator_name: "cpu"
          ptr: 82306880
        }
      }
    }
    timeline_label: "MatMul_10/a = Const()"
    scheduled_micros: 1516257747486272
    memory_stats {
    }
  }
  node_stats {
    node

In [33]:
tf.RunOptions?